In [1]:
import numpy as np
import nibabel as nib
import sklearn
import matplotlib.pyplot as plt

In [2]:

label = 'ZI'
hemi = 'R'
nsamples = 10000
path = '/home/ROBARTS/jlau/akhanfscratch/zona_diffparc/src/'

data = np.load(f'{path}/diffparc/group/group_space-snsx32_res-0.7_seed-{label}_hemi-{hemi}_targets-cortical_nsamples-{nsamples}_connMap.npz')


In [3]:
def save_label_nii (labels,mask,affine,out_nifti):
    labels_vol = np.zeros(mask.shape)
    labels_vol[mask > 0] = labels+1 #add a 1 so label 0 is diff from bgnd
    labels_nib = nib.Nifti1Image(labels_vol,affine)
    nib.save(labels_nib,out_nifti)
    

In [4]:
conn_group = data['conn_group']
mask = data['mask']
affine = data['affine']

#concat subjects
conn_group_m = np.moveaxis(conn_group,0,2)
conn_concat = conn_group_m.reshape([conn_group_m.shape[0],conn_group_m.shape[1]*conn_group_m.shape[2]])

In [5]:
#run spectral clustering

method = 'spectralcosine'

for k in np.arange(2,12):
    from sklearn.cluster import SpectralClustering
    clustering = SpectralClustering(n_clusters=k, assign_labels="discretize",random_state=0,affinity=dist).fit(conn_concat)
    save_label_nii(clustering.labels_,mask,affine,f'{path}/diffparc/group/groupconcat_label-{label}_hemi-{hemi}_method-{method}_k-{k}_cluslabels.nii.gz')
    

NameError: name 'dist' is not defined

In [ ]:
# t-SNE

from sklearn.manifold import TSNE
conn_embedded = sklearn.manifold.TSNE(n_components=2,metric='cosine',random_state=100).fit_transform(conn_concat)
plt.figure()
plt.scatter(conn_embedded[:,0],conn_embedded[:,1]);


for k in np.arange(2,12):
    from sklearn.cluster import SpectralClustering
    clustering = SpectralClustering(n_clusters=k, assign_labels="discretize",random_state=0,affinity=dist).fit(conn_concat)    
    plt.figure()
    plt.subplot(1,2,1)
    plt.scatter(conn_embedded[:,0],conn_embedded[:,1],c=clustering.labels_);
    plt.title(f'spectral clustering k={k}, vis with t-SNE')


    #centroid,labels = sklearn.cluster.KMeans(n_clusters=k,random_state=100).fit_transform(conn_embedded)
    kmeans = sklearn.cluster.k_means(conn_embedded,k)
    plt.subplot(1,2,2)
    plt.scatter(conn_embedded[:,0],conn_embedded[:,1],c=kmeans[1])
    plt.title(f'k-means on t-SNE embedding, k={k}')

    method='TSNEKmeans'
    save_label_nii(kmeans[1],mask,affine,f'diffparc/group/groupconcat_label-{label}_hemi-{hemi}_method-{method}_k-{k}_cluslabels.nii.gz')


# Clustering left and right simultaneously

In [ ]:
# now, try clustering left and right at the same time

label = 'ZI'
nsamples = 10000

hemi = 'R'
data_R = np.load(f'diffparc/group/group_space-snsx32_res-0.7_seed-{label}_hemi-{hemi}_targets-cortical_nsamples-{nsamples}_connMap.npz')

hemi = 'L'
data_L = np.load(f'diffparc/group/group_space-snsx32_res-0.7_seed-{label}_hemi-{hemi}_targets-cortical_nsamples-{nsamples}_connMap.npz')


In [ ]:

#concat subjects
conn_group = data_L['conn_group']
mask_L = data_L['mask']
conn_group_m = np.moveaxis(conn_group,0,2)
conn_concat_L = conn_group_m.reshape([conn_group_m.shape[0],conn_group_m.shape[1]*conn_group_m.shape[2]])

conn_group = data_R['conn_group']
mask_R = data_R['mask']
conn_group_m = np.moveaxis(conn_group,0,2)
conn_concat_R = conn_group_m.reshape([conn_group_m.shape[0],conn_group_m.shape[1]*conn_group_m.shape[2]])

num_L = conn_concat_L.shape[0]
conn_concat = np.vstack((conn_concat_L,conn_concat_R))


In [ ]:
def save_label_LR_nii (labels,num_L,mask_L,mask_R,affine,out_nifti):
    labels_vol = np.zeros(mask_L.shape)
    labels_vol[mask_L > 0] = labels[:num_L]+1 #add a 1 so label 0 is diff from bgnd
    labels_vol[mask_R > 0] = labels[num_L:]+1 #add a 1 so label 0 is diff from bgnd
    labels_nib = nib.Nifti1Image(labels_vol,affine)
    nib.save(labels_nib,out_nifti)
    

In [ ]:
#run spectral clustering

method = 'spectralcosine'

for k in np.arange(2,12):
    from sklearn.cluster import SpectralClustering
    clustering = SpectralClustering(n_clusters=k, assign_labels="discretize",random_state=0,affinity=dist).fit(conn_concat)
    save_label_LR_nii(clustering.labels_,num_L,mask_L,mask_R,affine,f'diffparc/group/groupconcat_label-{label}_method-{method}_k-{k}_cluslabels.nii.gz')
    